# Creating count arrays.

**Timing: 1 hour**

Before the analysis of translation limitation can begin the data from the ribosome profiling experiments must be organized into count arrays. Count arrays are basically lists that record the number of reads which map to each base pair or codon position along a transcript. The count arrays will be created inside of a Jupyter notebook which is running inside of the Plastid Conda environment set up in the Plastid and Python environment preparations section. Using Plastid to create the count arrays will allow for important adjustments to be made to the data such as applying the p-site offsets made in the Determining p-site offsets section and sub-setting the data to only look at the coding regions of the transcripts. The count arrays will be saved as simple csv tables which can be easily incorporated into further analyses in later sections. 

In [1]:
import sys
sys.path.append("../Python_scripts")

## Step 19
Load in the python libraries and functions necessary for this pipeline. This includes several functions from plastid and the contents of our utilities.py file:

In [2]:
from plastid import BAMGenomeArray, GTF2_TranscriptAssembler, Transcript
import numpy as np
import pandas as pd
from plastid.plotting.plots import *
import utilities as utils
import matplotlib.pyplot as plt
from matplotlib.pyplot import figure
%matplotlib inline

## Step 20
Load in the table of P-site offsets created in the Determining p-site offsets section using the Pandas function read_csv:

In [3]:
# Load in the P-site offsets for condition 1
p_offsets_cond1 = pd.read_csv("../Datasets/testing_Psite_offsets/condition1_RPF_1_Aligned.toTranscriptome.out_p-site-offsets", sep="\t")

# Load in the P-site offsets for condition 2
p_offsets_cond2 = pd.read_csv("../Datasets/testing_Psite_offsets/condition2_RPF_1_Aligned.toTranscriptome.out_p-site-offsets", sep="\t")

## Step 21
Load in a GTF genome annotation file into python using Plastid’s GTF2_TranscriptAssembler function. This function will load in the transcripts as an iterator of Plastid’s transcript type objects which we will then convert to a list using Python’s list function:

In [4]:
# Load in the transcript information
transcripts = list(GTF2_TranscriptAssembler(open("../Datasets/reference_files/annotation.gtf"), return_type=Transcript))

## Step 22
Load in the Bam file containing the Ribosome Profiling data as a Bam Genome Array using Plastid’s BamGenomeArray() function and map the reads to their corresponding P-sites via the VariableThreePrimeMapFactory custom function in utilities.py and Plastid’s set_mapping function:

In [5]:
# Load in the alignments from both the condition 1 and condition 2 datasets
alignments_cond1 = BAMGenomeArray("../Datasets/testing_genome_alignments/condition1_RPF_1_Aligned.sortedByCoord.out.bam")
alignments_cond2 = BAMGenomeArray("../Datasets/testing_genome_alignments/condition2_RPF_1_Aligned.sortedByCoord.out.bam")

"/home/keeganfl/Desktop/Work_Fall_2021/Protocol_test/seleno_seq/"
# Set the P-site offset mappings for both datasets
alignments_cond1.set_mapping(utils.VariableThreePrimeMapFactory(p_offsets = p_offsets_cond1))
alignments_cond2.set_mapping(utils.VariableThreePrimeMapFactory(p_offsets = p_offsets_cond2))

## Step 23
For each transcript object in our list use Plastid’s get_counts function to create a numpy array that contains the number of counts at each position in the transcript:

In [6]:
# Initialize two lists to contain the count arrays
count_arrays_cond1 = []
count_arrays_cond2 = []

# Iterate through each transcript in the GTF file and extract the count arrays for that transcript from the alignments.
for transcript in transcripts:
    count_arrays_cond1.append(
      transcript.get_counts(alignments_cond1))
    count_arrays_cond2.append(
      transcript.get_counts(alignments_cond2))

## Step 24
Once the count arrays have been created the information on CDS regions contained in the transcript type objects can be used to alter the count arrays to only cover the CDS regions: 

In [7]:
# Initialize two lists which will contain the start and end positions of the CDS region of each transcript. 
cds_starts = []
cds_ends = []

# Iterate through each transcript and add the cds information to the lists
for transcript in transcripts:
    cds_starts.append(transcript.cds_start)
    cds_ends.append(transcript.cds_end)

# subset each count array to only look at the cds region. 
for i in range(len(count_arrays_cond1)):
    count_arrays_cond1[i] = list(count_arrays_cond1[i][cds_starts[i]:cds_ends[i]])
    count_arrays_cond2[i] = list(count_arrays_cond2[i][cds_starts[i]:cds_ends[i]])

## Step 25
Use the add_gene_ids function from utilities.py to append the transcript ID and gene ID of each transcript to the start of the count array:

In [8]:
utils.add_gene_ids(transcripts, count_arrays_cond1)
utils.add_gene_ids(transcripts, count_arrays_cond2)

## Step 26
Filter out any count arrays that are of insufficient length or have insufficient read density. In this example, count arrays which were under 200 base pairs in length or which had a read density cut-off below 0.15 reads per base pair were filtered out:

In [9]:
# Initialize two lists to hold the filtered arrays
filtered_array_cond1 = []
filtered_array_cond2 = []

# Iterate through each of the count arrays and save any count arrays that pass our filtering parameters
for array_1, array_2 in zip(count_arrays_cond1, count_arrays_cond2):
     if len(array_1) > 200 and sum(array_1 [2:])/len(array_1 [2:]) > 0.15 and sum(array_2 [2:])/len(array_2 [2:]) > 0.15:
            filtered_array_cond1.append(array_1)
            filtered_array_cond2.append(array_2)

## Step 27
Save the count arrays to be used in future notebooks. Use the custom save_count_positions function from utilities.py so that the count arrays are saved with a header that describes each column which it is easier to read. Troubleshooting 3.

In [10]:
utils.save_count_positions(filtered_array_cond1, "../Datasets/testing_count_arrays/condition1_1_counts.csv")
utils.save_count_positions(filtered_array_cond2, "../Datasets/testing_count_arrays/condition2_1_counts.csv")